In [193]:
import datetime
import hyperopt
import itertools
import math
import numpy
import operator
import pandas
import random
import requests
from IPython.display import Audio
from sklearn import (
    calibration,
    ensemble,
    linear_model,
    metrics,
    model_selection,
    naive_bayes,
    neighbors,
    neural_network,
    svm,
)
import time
import tqdm
import xgboost

### Read and Filter

In [207]:
dtype = {'smoke': float, 'alco': float, 'active': float}

def fix_ap(value):
    value = abs(value)
    while value > 500.0:
        value /= 10.0
    return value

def read_csv(filename):
    frame = pandas.read_csv(filename, sep=';', header=0, na_values='None', dtype=dtype).drop(['id'], axis=1)
    
    frame = pandas.get_dummies(frame, columns=[
        # 'smoke',
        # 'active',
        # 'alco',
        # 'gender',
        # 'cholesterol',
        # 'gluc',
    ])
    frame = frame.assign(
        bmi=(frame['weight'] / frame['height'] ** 2),
        # aged_smoke_0=(frame['smoke_0.0'] * frame['age']),
        # aged_smoke_1=(frame['smoke_1.0'] * frame['age']),
        # aged_active_0=(frame['active_0.0'] * frame['age']),
        # aged_active_1=(frame['active_1.0'] * frame['age']),
        # aged_alco_0=(frame['alco_0.0'] * frame['age']),
        # aged_alco_1=(frame['alco_1.0'] * frame['age']),
        # smoke_1_active_0=(frame['smoke_1.0'] * frame['active_0.0']),
    )
    frame = frame.drop([
        # 'smoke',
        # 'alco',
        # 'active',
    ], axis=1)
    
    frame['ap_hi'] = frame['ap_hi'].apply(fix_ap)
    
    return frame

In [208]:
train = read_csv('train.csv')

X = train.drop('cardio', axis=1).values
y = train['cardio'].values
print(f'X: {X.shape}')
print(f'y: {y.shape}')

X: (70000, 12)
y: (70000,)


### Helpers

In [355]:
def print_importances(estimator):
    for name, importance in sorted(zip(test, estimator.feature_importances_), key=operator.itemgetter(1), reverse=True):
        print(f'{name}: {importance:.7f}')

def make_int_grid(params, param_name, delta):
    n = params.get(param_name, 1)
    return {param_name: list(range(max(1, n - delta), n + delta + 1))}

def make_float_grid(params, param_name, delta, min_, max_, default=0.0):
    x = params.get(param_name, default)
    return {param_name: list({max(min_, x - delta), x, min(max_, x + delta)})}

def search(estimator, X, y, params, **param_grid):
    start_time = time.time()
    grid_search = model_selection.GridSearchCV(
        estimator.set_params(**params),
        param_grid,
        scoring='neg_log_loss',
        refit=False,
        cv=cv(),
    ).fit(X, y)
    params.update(grid_search.best_params_)
    print(f'[{(time.time() - start_time):.0f}s] {grid_search.best_score_:.7f} {grid_search.best_params_} | {param_grid}')

def search_xgb(X, y, params):
    estimator = xgboost.XGBClassifier(nthread=5, seed=0)
    for _ in itertools.count(start=1):
        print()
        searches = [
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'colsample_bytree', 0.01, 0.5, 1.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'subsample', 0.01, 0.5, 1.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'base_score', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'scale_pos_weight', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'reg_lambda', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'reg_alpha', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_float_grid(params, 'gamma', 0.01, 0.0, 1.0)),
            lambda: search(estimator, X, y, params, **make_int_grid(params, 'max_depth', 1)),
            lambda: search(estimator, X, y, params, **make_int_grid(params, 'n_estimators', 3)),
        ]
        random.shuffle(searches)
        last_params = dict(params)
        for _search in searches:
            _search()
        if params == last_params:
            break
    
    print('---------------------------------------------------')
    print(params)
    return estimator.set_params(**params).fit(X, y)

### CV

In [213]:
def cv():
    return model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

### Predict `smoke`, `alco` and `active`

In [296]:
X_helper = train.drop(['smoke', 'alco', 'active', 'cardio'], axis=1).values

y_smoke = train['smoke'].values
y_alco = train['alco'].values
y_active = train['active'].values

In [360]:
# smoke_params = {'base_score': 0.54, 'scale_pos_weight': 1.0, 'reg_lambda': 0.97, 'reg_alpha': 0.03, 'gamma': 0.22000000000000003, 'max_depth': 3, 'n_estimators': 76, 'subsample': 0.5, 'colsample_bytree': 0.8599999999999999}
# alco_params = {}
# active_params = {}


In [356]:
smoke_estimator = search_xgb(X_helper, y_smoke, smoke_params)


[36s] -0.2388175 {'max_depth': 3} | {'max_depth': [2, 3, 4]}
[36s] -0.2388175 {'reg_lambda': 0.97} | {'reg_lambda': [0.96, 0.97, 0.98]}
[36s] -0.2388175 {'reg_alpha': 0.019999999999999997} | {'reg_alpha': [0.019999999999999997, 0.009999999999999997, 0.03]}
[35s] -0.2388175 {'colsample_bytree': 0.9299999999999999} | {'colsample_bytree': [0.9299999999999999, 0.94, 0.95]}
[83s] -0.2388175 {'n_estimators': 79} | {'n_estimators': [76, 77, 78, 79, 80, 81, 82]}
[36s] -0.2388175 {'base_score': 0.53} | {'base_score': [0.52, 0.53, 0.54]}
[36s] -0.2388175 {'gamma': 0.2} | {'gamma': [0.19, 0.2, 0.21000000000000002]}
[36s] -0.2388175 {'subsample': 0.5} | {'subsample': [0.49, 0.5, 0.51]}
[24s] -0.2388175 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.99, 1.0]}

[36s] -0.2388175 {'reg_alpha': 0.019999999999999997} | {'reg_alpha': [0.019999999999999997, 0.009999999999999997, 0.03]}
[36s] -0.2388175 {'max_depth': 3} | {'max_depth': [2, 3, 4]}
[36s] -0.2388175 {'reg_lambda': 0.97} | {'reg_lambda':

KeyboardInterrupt: 

In [ ]:
alco_estimator = search_xgb(X_helper, y_alco, alco_params)

In [ ]:
active_estimator = search_xgb(X_helper, y_active, active_params)

### Predict `cardio`

In [292]:
# cardio_params = {'base_score': 0.51, 'gamma': 0.18, 'max_depth': 5, 'n_estimators': 80, 'reg_alpha': 0.019999999999999997, 'reg_lambda': 0.99, 'scale_pos_weight': 1.0}

In [ ]:
cardio_estimator = search_xgb(X, y, cardio_params)
print_importances(cardio_estimator)

In [ ]:
test = read_csv('test.csv')
X_helper_test = test.drop(['smoke', 'alco', 'active'], axis=1).values

# TODO: try with normal predict.
test['smoke'].fillna(pandas.Series(smoke_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)
test['alco'].fillna(pandas.Series(alco_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)
test['active'].fillna(pandas.Series(active_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)

numpy.savetxt(f'xgboost.txt', cardio_estimator.predict_proba(test.values)[:, 1], fmt='%f')